# python 单元测试基础手册

- 基于测试的开发: 
    1. 编写测试并使测试失败
    2. 实现功能
    3. 使测试通过 
- 测试驱动开发: 
    1. 编写测试并使测试失败
    2. 实现功能
    3. 使测试通过
    4. 重构代码，保证测试通过的情况下提高代码的可读性可维护性等

## 1. 测试的基础：使用断言


In [5]:
from typing import Union

class AccessRules:
    @classmethod
    def is_leap_year(cls, year: Union[str, int]):
        """ 检验某个年份是否是闰年
            校验规则: 四年一润，百年不润，四百年再润

        Args:
            year (int): 要校验的年份
        
        Notices:
            1. 存在对传入参数的检测, 不允许传入参数类型为str且以0开头
            2. 当前没有对传入参数为负数的情况进行限制

        Returns:
            bool:   True  - 闰年
                    False - 平年
        """
        if isinstance(year, str):
            assert len(str(int(year))) == len(year), f"Invalid parameter: year, value: {year}"

        year = int(year)
        if year % 400 == 0:
            return True
        elif year % 100 == 0:
            return False
        elif year % 4 == 0:
            return True
        return False


### 假设要测试4种情况：
- 输入为int类型，且对应闰年
- 输入为int类型，且对应平年
- 输入为str类型，且对应闰年
- 输入为str类型，且对应平年

测试样例如下

In [ ]:
def test_is_leap_year_true_input_int():
    """ 输入为int类型，且对应闰年
        预期: 返回True
    """
    assert AccessRules.is_leap_year(2008) == True

def test_is_leap_year_false_input_int():
    """ 输入为int类型，且对应平年
        预期: 返回True
    """
    assert AccessRules.is_leap_year(2007) == False

def test_is_leap_year_false_input_str():
    """ 输入为str类型，且对应闰年
        预期: 返回True
    """
    assert AccessRules.is_leap_year("2007") == False

def test_is_leap_year_true_input_str():
    """ 输入为str类型，且对应平年
        预期: 返回True
    """
    assert AccessRules.is_leap_year("2008") == True

## 2. 多组测试数据的简洁写法
上述4个测试函数是根据函数传入参数的类型以及值是否对应闰年两两组合得到的四个测试样例，也分别对应四个测试等价类<br>
由于测试需要划分测试等价类，于是会有多组测试用例. 比如对于一个判断分数是否大于等于60从而给出是否及格的判断函数，小于60分与大于等于60分就是该方法的测试等价类<br>
一个测试方法对于一个测试等价类得到的结果应该是相同的，而测试样例需要从每一个测试等价类中取一个具体的数据作为测试该等价类的测试用例<br><br>
上述is_leap_year的测试等价类自然不止4种，但划分等价测试类不是我们这部分要讨论的重点<br>
这里要讨论的是对于此类传入参数极其简单，测试描述可以完全直接反映在测试函数名称，测试数据以及测试预期结果的情况，有如下方法可以简化代码<br>
### 2.1 将多个测试数据写在函数中, 以遍历的方式逐一调用测试函数进行测试<br>
（极其不推荐，一个测试函数中最好只有一个assert, 否则测试不通过时定位错误需要额外定位是哪组测试数据通过）<br>
（适合一个测试函数多个assert的情况：为提高可读性将一个assert拆分成多个assert，本质上是对一次函数测试进行多层次的assert）

In [10]:
def test_is_leap_year():
    test_data = (
        (2008, True),
        (2007, False),
        ("2008", True),
        ("2007", False),
    )
    for test_input, expected_result in test_data:
        assert AccessRules.is_leap_year(test_input) == expected_result

### 2.2 使用pytest框架的parameterize装饰器<br>
（与方法1的差别在于，pytest会自动对测试数据进行分拆且以类似测试函数后缀的形式区分不同样例， 同时可以通过传入ids参数给@pytest.mark.parametrize()为用例名称赋值）

In [11]:
import pytest

@pytest.mark.parametrize("test_year,expected_result", [("2000", True),
                                                       ("2008", True),
                                                       (2008, True),
                                                       (400, True),
                                                       ("1900", False)])
def test_is_leapyear(test_year, expected_result):
    assert AccessRules.is_leap_year(test_year) == expected_result

## 3. 测试程序崩溃的情况
在上述is_leap_year()中存在一种异常情况，即当传入参数为字符串且以0开头时, 会直接抛出异常, 这种情况程序会在运行时报错.<br>
测试程序按预期抛出异常的方法如下:<br>
### 3.1 直接捕获异常, 进而验证异常是否为预期异常

In [22]:
def test_is_leapyear():
    try:
        AccessRules.is_leap_year("0400")
    except Exception as excep:
        assert isinstance(excep,AssertionError)
        assert str(excep) == "Invalid parameter: year, value: 0400"

### 3.2 使用unitest框架

In [27]:
import unittest

class TestExceptionCase(unittest.TestCase):
    def test_invalid_input(self):
        self.assertRaises(AssertionError, AccessRules.is_leap_year, "0400")
        self.assertRaisesRegex(AssertionError, "Invalid parameter: year, value: 0400", AccessRules.is_leap_year, "0400")

### 3.3 使用pytest框架

In [28]:
import pytest

@pytest.mark.xfail(raises=AssertionError)
def test_invalid_input():
    AccessRules.is_leap_year("0400")

def test_invalid_input_2():
    with pytest.raises(AssertionError, match='Invalid parameter: year, value: 0400'):
        AccessRules.is_leap_year("0400")

## 4. 组合技

In [ ]:
@pytest.mark.parametrize("test_year,expected_result", [("2000", True),
                                                       ("2008", True),
                                                       (2008, True),
                                                       (400, True),
                                                       (-400, True),
                                                       ("-400", True),
                                                       pytest.param("0400", False, marks=pytest.mark.xfail),
                                                       pytest.param("0400", True, marks=pytest.mark.xfail),
                                                       ("1900", False)])
def test_is_leapyear(test_year, expected_result):
    assert AccessRules.is_leap_year(test_year) == expected_result


## 5. 使用Mock对象及方法以解除外部依赖（如网页接口或数据库请求，Redis请求，时间戳生成，随机数生成等）
### 5.1 对Request请求进行Mock（以httpbin.org作为辅助进行说明）
以下例程为不进行Mock，完全依赖于httpbin.org这个网站返回情况的情况

In [53]:
"""
对https://httpbin.org/get进行请求会返回所有请求中的参数，在返回体的args中
https://httpbin.org/get?text=Hello World的返回如下
{
    "args": {
        "text": "Hello World"
    }, 
    "headers": {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
        "Accept-Encoding": "gzip, deflate, br", 
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8", 
        "Host": "httpbin.org", 
        "Sec-Ch-Ua": "\"Google Chrome\";v=\"87\", \" Not;A Brand\";v=\"99\", \"Chromium\";v=\"87\"", 
        "Sec-Ch-Ua-Mobile": "?0", 
        "Sec-Fetch-Dest": "document", 
        "Sec-Fetch-Mode": "navigate", 
        "Sec-Fetch-Site": "none", 
        "Sec-Fetch-User": "?1", 
        "Upgrade-Insecure-Requests": "1", 
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36", 
        "X-Amzn-Trace-Id": "Root=1-6017d21d-395b4a050a9c34904fccf3ea"
    }, 
    "origin": "108.166.207.187", 
    "url": "https://httpbin.org/get?text=Hello World"
    }
"""

import requests

def test_request_parameters():
    parameters = {'Hello': 'World'}
    parameter_pairs = [f"{param_name}={param_value}" for param_name, param_value in parameters.items()]
    resp = requests.get(f"https://httpbin.org/get?{'&'.join(parameter_pairs)}")
    assert resp.status_code == 200, f"Expect Response Status Code 200, But Got {resp.status_code}"
    # assert resp.status_code == 500, f"Expect Response Status Code 500, But Got {resp.status_code}"
    assert resp.json()["args"] == parameters

def test_request_parameters_poor_readability():
    parameters = {'text': 'Hello World'}
    resp = requests.get(f"https://httpbin.org/get?{'&'.join([key+'='+value for key, value in parameters.items()])}")
    assert resp.json()["args"] == parameters
test_request_parameters()

上述测试的问题在于: **其Assert语句是否为真完全依赖于httpbin.org的实际返回**<br>
当我们希望测试请求成功，返回状态码为200的情况时，我们写下<br>
```assert resp.status_code == 200, f"Expect Response Status Code 200, But Got {resp.status_code}"```<br>
当该网站服务正常时，我们的测试将顺利通过。但当网站的服务出现异常的时候，我们的单元测试将不通过。<br>
从另外一个角度，如果我们试图测试返回异常的情况，我们写下<br>
```assert resp.status_code == 500, f"Expect Response Status Code 500, But Got {resp.status_code}"```<br>
则该测试只能在网站请求异常时通过。<br>
而我们希望的时我们的测试在任何时间，任何物理地点以及任何软件环境下能有相同的行为，能保证其测试的独立性, 于是我们需要使用Mock来实现这一点

### 5.2 Introduction to Mock
顾名思义，mock的本质是通过模仿指定对象的行为来实现对其解除依赖的目的。我们能自由定义用来替换外部依赖的mock对象的一切行为，包括且不限于属性，方法，方法的返回值。

In [112]:
from unittest import mock
mock_example = mock.Mock()
print(mock_example)
print(mock_example())

<Mock id='3170985473984'>
<Mock name='mock()' id='3170985474944'>


在python中, 利用unittest中的mock可以Mock出对象
在我们为对其行为进行定义前，其行为都为各种mock对象，如上述mock_example，我们未对其定义__call__方法而直接调用的话将返回另一个mock对象<br>
而我们可以对其行为进行如下定义: 

#### 定义mock对象被当作函数调用时的返回值

In [113]:
mock_example.return_value = "Result when call mock_example" 
mock_example()

'Result when call mock_example'

#### 定义Mock对象的属性

In [114]:
mock_example.status_code = 200
mock_example.status_code

200

#### 定义Mock对象的方法
需要注意的是, 由于我们定义的是方法，即当我们定义方法mock_method后，将以mock_method()的形式调用它, 这与刚才我们直接将mock对象当作函数调用时的形式一致。<br>
故我们刚才通过定义mock对象的return_value来定义函数返回值, 则此时也是通过定义mock_method.return_value来定义这个方法的返回值

In [115]:
mock_example.say_hi.return_value = "Hi!"
mock_example.say_hi()

'Hi!'

有了对mock的基本了解，重新分析5.1中模拟并替换requests库的get方法，则可以有如下实现:

In [120]:
import requests
from unittest import mock

def test_request_parameters_mock_get():
    parameters = {'text': 'Hello World'}
    mock_response = mock.Mock(status_code=200)
    mock_response.json.return_value = {"args": parameters}
    requests = mock.Mock()
    requests.get.return_value = mock_response
    parameter_pairs = [f"{param_name}={param_value}" for param_name, param_value in parameters.items()]
    resp = requests.get(f"https://httpbin.org/get?{'&'.join(parameter_pairs)}")
    assert resp.status_code == 200, f"Expect Response Status Code 200, But Got {resp.status_code}"
    assert resp.json()["args"] == parameters

利用pytest，我们可以提升一下代码的简洁性和可读性

In [108]:
import requests
from unittest import mock

""" usage: @mock.patch("the_import_path_of_the_object_you_wanna_mock")
    like: @mock.patch("src.utils_base.redis_client.RedisHub")       # 对应 from src.utils_base.redis_client import RedisHub
    like: @mock.patch("src.utils_base.redis_client.RedisHub.rpush") # 对应 from src.utils_base.redis_client import RedisHub; RedisHub.rpush()

    将用来以假乱真的mock对象的变量名作为传入参数添加进被测试函数. 如上一个代码块中的requests对象. 
    由于我们只需要用到requests的get方法, 可以直接mock该方法, 而不用去mock外层的requests库
    即我们将用requests_get来替换request.get方法, 以resp = requests_get(f"https://httpbin.org/get?{'&'.join(parameter_pairs)}")的形式调用

    为了避免混淆, 增加可读性, 我们增加一个mock_前缀给变量名来表示其性质, 故以下代码中变量名为mock_requests_get
"""

@mock.patch("requests.get")
def test_request_parameters_mock_get(mock_requests_get):
    parameters = {'text': 'Hello World'}
    mock_response = mock.Mock(status_code=200)
    mock_response.json.return_value = {"args": parameters}
    mock_requests_get.return_value = mock_response

    parameter_pairs = [f"{param_name}={param_value}" for param_name, param_value in parameters.items()]
    resp = requests.get(f"https://httpbin.org/get?{'&'.join(parameter_pairs)}")
    assert resp.status_code == 200, f"Expect Response Status Code 200, But Got {resp.status_code}"
    assert resp.json()["args"] == parameters


### 5.3 对当前IP或操作系统进行mock
假设我们有一个代码块需要对当前运行程序的环境进行判断并对应不同环境执行不同操作，如下:

In [135]:
import platform
import socket
import os

def detect_environment():
    ENV_DEV = "DEV"
    ENV_TEST = "TEST"
    ENV_PROD = "PROD"

    OS_TYPE = platform.system().upper()
    HOST_NAME = socket.gethostname()
    IP_ADDR = socket.gethostbyname(HOST_NAME)

    if OS_TYPE == "WINDOWS" or HOST_NAME.lower().startswith('万物皆可MOCK'):
        ENV = ENV_DEV
    elif HOST_NAME.lower().find("sit") > -1 or IP_ADDR == "10.80.62.161":
        ENV = ENV_TEST
    else:
        ENV = ENV_PROD
    return ENV

则在不进行mock的情况下, 我们的OS类型, 主机名, IP地址将根据实际情况变化<br>
比如当前我在Windows操作系统中, IP地址为192.168.21.84
则程序运行结果如下:

In [132]:
print(f"Current ENV: {detect_environment()}")

Current ENV: DEV


假设我们希望测试其他环境的程序行为，则可以通过mock操作系统, IP, 主机名来实现
<br>
#### 5.3.1 Mock操作系统

In [133]:
platform = mock.Mock()
platform.system.return_value = "linux"
print(platform.system())
print(platform.system().upper()) # uppper()方法是字符串自带的不需要MOCK
print(f"Current ENV: {detect_environment()}")

linux
LINUX
Current ENV: PROD


#### 5.3.2 Mock主机名
通过detect_environment()的实现逻辑, 我们知道就算当前的操作系统是linux, 但只要我们的主机名以"万物皆可MOCK"开头, 当前环境也会被识别为"DEV"<br>
于是，我们现在就将我们的主机名改为以"万物皆可MOCK"作为前缀的主机名

In [138]:
socket = mock.Mock()
socket.gethostname.return_value = "万物皆可MOCK，连MOCK都可以MOCK"
print(f"Current HostName: {socket.gethostname()}")
print(f"Current ENV: {detect_environment()}")

Current HostName: 万物皆可MOCK，连MOCK都可以MOCK
Current ENV: DEV


#### 5.3.3 Mock本机IP (做法相似，此处不再给出具体编码实现，给读者留作练习)

#### 5.3.4 从其他角度进行mock达到相同目的
如果我们只是希望detect_environment()能够将我们当前环境识别为TEST, 则除了mock当前IP、操作系统、主机名之外, 其实我们可以……直接mockdetect_environment, 令其return_value为"TEST"即可

### 5.4 对Redis请求进行Mock并根据参数的不同返回不同的结果
假设我们利用了redis来存储少数SVIP用户ID, 并在程序中通过检查用户ID是否存在于Redis中来决定是否执行SVIP行为, 如下:

In [150]:
import redis 

def check_whether_SVIP(user_id: str):
    redis_con = redis.StrictRedis(host="localhost", port=6039, db=0, password="nopassword")
    target_key = "the_place_stored_svip_id"
    if redis_con.sismember(target_key, user_id):
        print(f"{user_id} is SVIP.")
    else:
        print(f"{user_id} is not SVIP.")


目前为止, 我们mock函数或方法时都是定义其return_value使其在被调用时固定返回我们希望的值<br>
如果我们希望调用时根据传入参数的不同而有不同的返回，也即我们希望定义函数对参数进行判断后决定该返回何值时, 我们就可以使用「side_effect」, 如下:

In [153]:
import pytest
import redis

def test_SVIP():
    def judge_logic(user_id):
        return True if user_id == "ef30c771-eaa8-4bab-b666-8061eee53610" else False
    check_whether_SVIP = mock.Mock()
    check_whether_SVIP.side_effect = judge_logic
    assert check_whether_SVIP("ef30c771-eaa8-4bab-b666-8061eee53610") == True
    
test_check_whether_SVIP()

当然，也可以通过对redis连接以及redis对其数据结构的操作来达到我们的目的。但我们只希望能够在传入"ef30c771-eaa8-4bab-b666-8061eee53610"给check_whether_SVIP得到True，其他情况返回False, 只对check_whether_SVIP进行mock就足够了。

### 5.5 测试调用次数(或者redo次数)以及调用信息

In [163]:
import pytest
import redis

def test_SVIP():
    def judge_logic(user_id):
        return True if user_id == "ef30c771-eaa8-4bab-b666-8061eee53610" else False
    check_whether_SVIP = mock.Mock()
    check_whether_SVIP.side_effect = judge_logic
    check_whether_SVIP("ef30c771-eaa8-4bab-b666-8061eee53610")
    check_whether_SVIP("ef30c771-eaa8-4bab-b666-8061eee53610")
    check_whether_SVIP.side_effect.assert_called_once()
    assert check_whether_SVIP.side_effect.assert_called_once_with("XXXXXXXXXXX")
    
test_check_whether_SVIP()

In [202]:
def judge_logic(user_id):
        return True if user_id == "ef30c771-eaa8-4bab-b666-8061eee53610" else False
check_whether_SVIP = mock.Mock()
check_whether_SVIP.side_effect = judge_logic
check_whether_SVIP("ef30c771-eaa8-4bab-b666-8061eee53610")
check_whether_SVIP.assert_called_once()

In [203]:
""" 由于我们调用check_whether_SVIP()时传入参数为"ef30c771-eaa8-4bab-b666-8061eee53610"而不是"XXXXXXXX"
    所以当我们断言heck_whether_SVIP以参数为"XXXXXXXX"被调用时将抛出AssertError
"""
check_whether_SVIP.assert_called_once_with("XXXXXXXX")

AssertionError: expected call not found.
Expected: mock('XXXXXXXX')
Actual: mock('ef30c771-eaa8-4bab-b666-8061eee53610')

In [200]:
""" 在上一个cell中运行之后再运行该cell将报错, 因为此时check_whether_SVIP()被调用两次, 不满足assert_called_once
"""
check_whether_SVIP("ef30c771-eaa8-4bab-b666-8061eee53610")
check_whether_SVIP.assert_called_once()

AssertionError: Expected 'mock' to have been called once. Called 2 times.
Calls: [call('ef30c771-eaa8-4bab-b666-8061eee53610'),
 call.getdoc(),
 call('ef30c771-eaa8-4bab-b666-8061eee53610')].

In [ ]:
### 5.6 测试生成本地配置文件（利用fixture的tmpdir）

In [ ]:
### 5.7 更多玩法
- 测试区分python版本
- 